In [ ]:
# Q5. MNIST 데이터셋을 Load -> 학습할 모델을 구현, 가중치 초기화 
#     -> 구현한 모델을 학습시키기 위한 loss 함수 opitmizer 구현
#     -> 구현한 함수들을 이용해 학습 Loop 구현 
#     -> 학습 완료 후 테스트 데이터를 사용해 테스트 진행
# output : 테스트 코드

# A. 
#   1. outpus 구현

import torch
import torch.nn as nn
import torchvision.datasets as dset 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
 

training_epochs = 15 
batch_size = 100 

root = './data' 
mnist_train = dset.MNIST (root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST (root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True) 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias = True).to(device)

torch.nn.init.normal_(linear.weight)

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)
# lr = learning rate, 미분값을 얼만큼 이동시킬 것인가를 결정 한다.
# 초기값이 크다면 초반에는 loss값이 빠르게 줄어들지만 나중에는 underfitting이 발생하게 된다.

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
# 다중분류를 위한 손실함수, 다중 분류에 사용된다. 
# torch.nn.CrossEntropyLoss는 nnLogSoftMax와 nn.NLLLoss 연산의 조합이다. 
# 배치 처리에 대한 손실값은 배치를 구성하는 각 데이터 손실값의 평균이다. 
# 연산을 한번에 처리해 수식이 간소화 되어 역전파가 더 안정적으로 이루어지므로 위 코드와 같은 방법이 실제 사용에 권장된다.
# Softmax와 Log처리 및 CrossEntropy연산의 조합이다. 
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)
# parameter 최적화
# 최적화는 각 학습 단계에서 모델의 오류를 줄이기위해 모델 매개변수를 조정하는 과정이다. 
# 최적화 알고리즘은 이 과정에서 수행되는 방식을 정의한다. 
# 위 코드의 경우 Stochastic Gradient Descent, 즉 확률적 경사하강법을 정의한다.
# 학습하려는 모델의 매개변수와 학습률을 하이퍼파라미터를 등록해 optimizer를 초기화한다.
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(training_epochs):
  for i, (imgs, labels) in enumerate(train_loader):
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = imgs.view(-1, 28 * 28)

    outputs = linear(imgs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

    _,argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax).float().mean()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(
            epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))

In [ ]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax). sum().item()

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))